# Checks of cyclists data

In [1]:
# Autoreload allows the notebook to dynamically load code: if we update some helper functions *outside* of the notebook, we do not need to reload the notebook.
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import unicodedata

We load the dataset from a CSV file and display the first few rows to get an initial understanding of the data. This helps us verify that the data has been loaded correctly and gives us a glimpse of its structure and contents.

In [3]:
csv_file = "../data/cyclists.csv"
dataset = pd.read_csv(csv_file)
dataset.head()

,_url,name,birth_year,weight,height,nationality
0,bruno-surra,Bruno Surra,1964.0,NaN,NaN,Italy
1,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France
2,jan-maas,Jan Maas,1996.0,69.0,189.0,Netherlands
3,nathan-van-hooydonck,Nathan Van Hooydonck,1995.0,78.0,192.0,Belgium
4,jose-felix-parra,José Félix Parra,1997.0,55.0,171.0,Spain


Create dataset from the union of the cyclists and the races data 

In [32]:
# Create union of two datasets, merging them considering the url of the cyclist
dataset_cyclists = pd.read_csv("../data/cyclists.csv")
dataset_races = pd.read_csv("../data/races.csv")
merged_dataset = pd.merge(dataset_cyclists, dataset_races, left_on='_url', right_on='cyclist', how='inner')

# Modify name column of the cyclist url in '_url_cyclist', and name column of the race url in '_url_race'
merged_dataset = merged_dataset.rename(columns={'_url_x': '_url_cyclist', '_url_y': '_url_race'})
# Modify name column of the cyclist name in 'name_cyclist', and name column of the race name in 'name_race'
merged_dataset = merged_dataset.rename(columns={'name_x': 'name_cyclist', 'name_y': 'name_race'})
# Take only the year-month-day part of 'date' (delete the time)
merged_dataset['date'] = merged_dataset['date'].str.split(' ').str[0]

merged_dataset.head()


,_url_cyclist,name_cyclist,birth_year,weight,height,nationality,_url_race,name_race,points,uci_points,...,average_temperature,date,position,cyclist,cyclist_age,is_tarmac,is_cobbled,is_gravel,cyclist_team,delta
0,bruno-surra,Bruno Surra,1964.0,NaN,NaN,Italy,vuelta-a-espana/1989/stage-1,Vuelta a España,80.0,NaN,...,NaN,1989-04-24,110,bruno-surra,25.0,True,False,False,NaN,15.0
1,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France,tour-de-france/1997/stage-2,Tour de France,100.0,NaN,...,NaN,1997-07-07,132,gerard-rue,32.0,True,False,False,denmark-1991,0.0
2,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France,tour-de-france/1990/stage-1,Tour de France,100.0,NaN,...,NaN,1990-07-01,66,gerard-rue,25.0,True,False,False,france-1978,635.0
3,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France,tour-de-france/1992/stage-7,Tour de France,100.0,NaN,...,NaN,1992-07-11,35,gerard-rue,27.0,True,False,False,france-1978,65.0
4,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France,tour-de-france/1990/stage-9,Tour de France,100.0,NaN,...,NaN,1990-07-09,41,gerard-rue,25.0,True,False,False,france-1978,37.0


## Initial Info

Now we provide a concise summary of the DataFrame, including the number of non-null entries, data types of each column, and memory usage. It helps us quickly identify missing values and understand the overall structure of the dataset.

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _url         6134 non-null   object 
 1   name         6134 non-null   object 
 2   birth_year   6121 non-null   float64
 3   weight       3078 non-null   float64
 4   height       3143 non-null   float64
 5   nationality  6133 non-null   object 
dtypes: float64(3), object(3)
memory usage: 287.7+ KB


Also, we generates a descriptive statistics for numerical columns in the DataFrame. It includes metrics such as count, mean, standard deviation, minimum, and maximum values, as well as the 25th, 50th, and 75th percentiles. This summary helps us understand the distribution and central tendency of the data.

In [5]:
dataset.describe()

,birth_year,weight,height
count,6121.000000,3078.000000,3143.000000
mean,1974.071884,68.658739,179.815145
std,15.535834,6.348183,6.443447
min,1933.000000,48.000000,154.000000
25%,1962.000000,64.000000,175.000000
50%,1974.000000,69.000000,180.000000
75%,1987.000000,73.000000,184.000000
max,2004.000000,94.000000,204.000000


## Check on '_url' data

We start considering the `_url` column, and check the number of null values and the count the occurrences of each unique value

In [7]:
print('Total number of null values in _url column: ' + str(dataset['_url'].isnull().sum())
      + ' (' + str(round(dataset['_url'].isnull().sum() / len(dataset) * 100, 2)) + '%)')

print('\nCount occurrences of each value in _url column:')
url_counts = dataset['_url'].value_counts()
print(url_counts)

Total number of null values in _url column: 0 (0.0%)

Count occurrences of each value in _url column:
_url
ward-vanhoof            1
bruno-surra             1
gerard-rue              1
jan-maas                1
nathan-van-hooydonck    1
                       ..
stian-remme             1
scott-davies            1
joost-van-leijen        1
chad-haga               1
willy-moonen            1
Name: count, Length: 6134, dtype: int64


We have lots different values, but no null values.

## Check on 'name' data

Now we consider the `name` column, and check the number of null values and the count the occurrences of each unique value

In [10]:
print('Total number of null values in name column: ' + str(dataset['name'].isnull().sum())
        + ' (' + str(round(dataset['name'].isnull().sum() / len(dataset) * 100, 2)) + '%)')

print('\nCount occurrences of each value in name column:')
name_counts = dataset['name'].value_counts()
print(name_counts)   

Total number of null values in name column: 0 (0.0%)

Count occurrences of each value in name column:
name
Sergio  Domínguez       2
Alberto  Fernández      2
Jesús  López            2
Antonio  Cabello        2
Alessandro  Pozzi       2
                       ..
Juan José  Martínez     1
Iñigo  Elosegui         1
Paolo  Alberati         1
Jackson  Rodríguez      1
Jean-Philippe  Dojwa    1
Name: count, Length: 6127, dtype: int64


We have lots of different values, but no null values.

Since we have a lot of different values, we check if every value is sintatically correct

In [37]:
# For each data, check if 'name' object contains any number
# Check if the 'name' column contains any number
invalid_rows = dataset[dataset['name'].str.contains(r'\d')]

print('Number of rows with numbers in the name: ' + str(len(invalid_rows)))

Number of rows with numbers in the name: 0


## Check on 'birth_year' data

Now we consider the `birth_year` column, and check the number of null values and the count the occurrences of each unique value

In [13]:
print('Total number of null values in birth_year column: ' + str(dataset['birth_year'].isnull().sum())
        + ' (' + str(round(dataset['birth_year'].isnull().sum() / len(dataset) * 100, 2)) + '%)')

print('\nCount occurrences of each value in birth_year column:')
birth_year_counts = dataset['birth_year'].value_counts()
print(birth_year_counts)

Total number of null values in birth_year column: 13 (0.21%)

Count occurrences of each value in birth_year column:
birth_year
1964.0    145
1962.0    141
1970.0    140
1974.0    138
1980.0    133
         ... 
1937.0      4
1934.0      2
1938.0      2
1933.0      1
1936.0      1
Name: count, Length: 71, dtype: int64


We have different values, and a few null values

Since we have different values, we check if every value is sintatically correct

In [39]:
# Get rows where 'birth_year' does not end with '.0'
invalid_rows = dataset[~dataset['birth_year'].astype(str).str.endswith('.0')].dropna(subset=['birth_year'])

print('Number of invalid birth_year: ' + str(len(invalid_rows)))

Number of invalid birth_year: 0


In this block we check if the `birth year` value is not in the form 'nnnn' and if it is not in the form '19nn' or '20nn'

In [41]:
# Remove '.0' from 'birth_year' and check if it's a valid year in the form '19nn' or '20nn'
def is_valid_year(birth_year):
    # Convert to string and remove '.0'
    year = str(birth_year).replace('.0', '')
    # Check if the year is a digit and starts with '19' or '20'
    return year.isdigit() and (year.startswith('19') or year.startswith('20'))

# Filter out rows where 'birth_year' is not null
invalid_rows = dataset[dataset['birth_year'].notnull()]

# Apply the validation function across the 'birth_year' column
invalid_rows = invalid_rows[~invalid_rows['birth_year'].apply(is_valid_year)]

print('Number of invalid birth_year: ' + str(len(invalid_rows)))


Number of invalid birth_year: 0


Check the races where the `birth_year` value is small or large, for possible outliers

In [16]:
# Dataset info, for 'birth_year' column
dataset['birth_year'].describe()

count    6121.000000
mean     1974.071884
std        15.535834
min      1933.000000
25%      1962.000000
50%      1974.000000
75%      1987.000000
max      2004.000000
Name: birth_year, dtype: float64

The birth years are between 1933 and 2004, so we can say that no extreme values are present (in the cyclism context).

## Check on 'weight' data

Now we consider the `weight` column, and check the number of null values and the count the occurrences of each unique value

In [19]:
print('Total number of null values in weight column: ' + str(dataset['weight'].isnull().sum())
        + ' (' + str(round(dataset['weight'].isnull().sum() / len(dataset) * 100, 2)) + '%)')

print('\nCount occurrences of each value in weight column:')
weight_counts = dataset['weight'].value_counts()
print(weight_counts)

Total number of null values in weight column: 3056 (49.82%)

Count occurrences of each value in weight column:
weight
70.0    272
68.0    219
65.0    193
67.0    177
72.0    169
69.0    162
73.0    146
63.0    140
66.0    139
64.0    137
74.0    135
62.0    131
75.0    128
71.0    125
60.0     98
61.0     90
78.0     86
77.0     67
58.0     64
76.0     63
80.0     53
59.0     49
79.0     30
82.0     26
55.0     25
81.0     22
83.0     20
57.0     20
56.0     19
85.0     10
53.0      7
52.0      6
84.0      6
54.0      4
51.0      4
90.0      4
87.0      3
88.0      3
63.5      2
89.0      2
50.0      2
58.5      2
86.0      2
71.5      1
48.0      1
91.0      1
67.5      1
66.5      1
78.1      1
77.5      1
74.5      1
81.4      1
62.5      1
93.0      1
73.5      1
79.5      1
65.1      1
92.0      1
94.0      1
Name: count, dtype: int64


We have different values, but a lot of null values. Also, we see that every value is sintatically correct

Check the races where the `weight` value is small or large, for possible outliers

In [20]:
# Dataset info, for 'weight' column
dataset['weight'].describe()

count    3078.000000
mean       68.658739
std         6.348183
min        48.000000
25%        64.000000
50%        69.000000
75%        73.000000
max        94.000000
Name: weight, dtype: float64

The weights are between 48 and 94 kg, so we can say that no extreme values are present (in the cyclism context).

## Check on 'height' data

Now we consider the `height` column, and check the number of null values and the count the occurrences of each unique value

In [23]:
print('Total number of null values in height column: ' + str(dataset['height'].isnull().sum())
        + ' (' + str(round(dataset['height'].isnull().sum() / len(dataset) * 100, 2)) + '%)')

print('\nCount occurrences of each value in height column:')
height_counts = dataset['height'].value_counts()
print(height_counts)

Total number of null values in height column: 2991 (48.76%)

Count occurrences of each value in height column:
height
180.0    277
178.0    226
183.0    193
181.0    181
175.0    169
182.0    165
185.0    161
176.0    154
184.0    152
179.0    137
177.0    133
174.0    129
173.0    120
186.0    107
190.0     97
170.0     90
187.0     85
172.0     80
188.0     77
171.0     67
189.0     48
169.0     46
191.0     37
192.0     34
168.0     24
167.0     23
193.0     22
164.0     20
194.0     17
195.0     13
165.0     13
196.0      7
197.0      6
166.0      6
198.0      4
160.0      4
162.0      3
159.0      3
161.0      2
199.0      2
163.0      2
154.0      1
204.0      1
155.0      1
158.0      1
202.0      1
157.0      1
200.0      1
Name: count, dtype: int64


We have different values, but a lot of null values. Also, we see that every value is sintatically correct

Check the races where the `height` value is small or large, for possible outliers

In [24]:
# Dataset info, for 'height' column
dataset['height'].describe()

count    3143.000000
mean      179.815145
std         6.443447
min       154.000000
25%       175.000000
50%       180.000000
75%       184.000000
max       204.000000
Name: height, dtype: float64

The heights are between 154 and 204 cm, so we can say that no extreme values are present (in the cyclism context).

## Check on 'nationality' data

Now we consider the `nationality` column, and check the number of null values and the count the occurrences of each unique value

In [30]:
print('Total number of null values in nationality column: ' + str(dataset['nationality'].isnull().sum())
        + ' (' + str(round(dataset['nationality'].isnull().sum() / len(dataset) * 100, 2)) + '%)')

print('\nCount occurrences of each value in nationality column:')
nationality_counts = dataset['nationality'].value_counts()
print(nationality_counts)

Total number of null values in nationality column: 1 (0.02%)

Count occurrences of each value in nationality column:
nationality
Italy                 1029
Spain                  872
Belgium                869
France                 741
Netherlands            380
                      ... 
Dominican Republic       1
Liechtenstein            1
Zimbabwe                 1
Puerto Rico              1
Hongkong                 1
Name: count, Length: 72, dtype: int64


We have different values, and just one null value.

Since we have a lot of different values, we check if every value is sintatically correct

In this block we check if there are `nationality` values that contains any character that is a letter

In [47]:
# Check if there are 'nationality' values that contains any number, without the null values
invalid_rows = dataset[dataset['nationality'].notnull() & dataset['nationality'].str.contains(r'\d')]

print('Number of rows with numbers in nationality: ' + str(len(invalid_rows)))

Number of rows with numbers in nationality: 0
